<a href="https://colab.research.google.com/github/emerenan/validador/blob/master/validador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

def readfile(path):
    """ Método para leitura de arquivo no formato .csv
    
    Arg: \n
        path: caminho onde o arquivo está localizado. \n
    Returns: \n
        Retorna um dataframe com as informações do 
    """
    try:
        df = pd.read_csv(path, sep=';', error_bad_lines=False, warn_bad_lines=True, encoding='U8')
        # print(df.head())
        # print('-----------------------------------------------------------------------')
        # print(df.isnull().sum())
        # print('-----------------------------------------------------------------------')
        # print(df.info())
        # print('-----------------------------------------------------------------------')
        return df
    
    except pd.io.common.ParserError:
        print("Dados não lidos")

def contador(df, coluna):
    """ Método para contagem de valores na coluna, mostrando ou não os valores repetidos.
    Arg: \n
        df: nome da dataframe que será usado para contagem. \n
        coluna: nome da coluna que deverá ser feita a contagem dos valores. \n

    Returns: \n
        Retorna um dicionario com a contagem do valores, assim como os repetidos.\n

    """
    # dicionário para salvar os valores contados
    index_cont = {}

    # Extraindo os valores da coluna e salvando na variável
    col = df[coluna]

    # Iterando entre a coluna do dataframe
    for index in col:
        # Se o index for existente, add 1
        if index in index_cont.keys():
            index_cont[index] += 1
        else:
            index_cont[index] = 1
    
    # return index_cont
    # Dicionario para contar os valores sem repetição e com repetição
    rep = {'sem_repeticao' : 0, 'com_repeticao' : 0}

    # Iterando sobre o dicionario index_col e contando os valores sem repetição e com repetição
    for key, value in index_cont.items():
        if value == 1:
            rep['sem_repeticao'] += 1
        else:
            rep['com_repeticao'] += 1
    
    return rep

data_be = readfile('/content/310522_BE_160420.csv')
data_co = readfile('/content/310522_CO_012020_072020.csv')
# data_si = readfile('/content/310522_SI_012020_062020.csv')
# data_pl = readfile('/content/310522_PL_08052020.csv')
# data_pr = readfile('/content/310522_PR_052020.csv')
# data_pp = readfile('/content/310522_PP_201912.csv')
# data_pc = readfile('/content/310522_PC_08052020.csv')

data_be = data_be.fillna(0)
data_be[['CD_MATRICULA_TEM', 'DT_EXCLUSAO', 'DT_CPT']] = data_be[['CD_MATRICULA_TEM', 'DT_EXCLUSAO', 'DT_CPT']].astype('int')

teste = contador(data_co, 'CD_MATRICULA_PRINCIPAL')
print(teste)









   

{'sem_repeticao': 223, 'com_repeticao': 20660}


In [ ]:
for coluna, linha in data_be.iterrows():
    if data_be.loc[coluna, 'CD_MATRICULA'] == data_be.loc[coluna, 'CD_MATRICULA_TEM'] :
        cont['ativo'] += 1
    elif data_be.loc[coluna, 'CD_MATRICULA'] != data_be.loc[coluna, 'CD_MATRICULA_TEM']:
        cont['ativo'] += 2
    else:
        cont['inativo'] += 1

print(cont)
"""
ativo = 0
inativo = 0
for label, row in benef.iterrows():
    if benef.loc[label, 'DT_EXCLUSAO'] == 0:
        ativo += 1
    else:
        inativo += 1

print("Ativos: "+ str(ativo) +"\n"+ "Inativos: "+ str(inativo))
"""

In [105]:
import pandas as pd
import numpy as np

def readfile(path):
    """ Método para leitura de arquivo no formato .csv
    
    Arg: \n
        path: caminho onde o arquivo está localizado. \n
    Returns: \n
        Retorna um dataframe com as informações do 
    """
    try:
        df = pd.read_csv(path, sep=';', error_bad_lines=False, warn_bad_lines=True)
        # print(df.head())
        # print('-----------------------------------------------------------------------')
        # print(df.isnull().sum())
        # print('-----------------------------------------------------------------------')
        # print(df.info())
        # print('-----------------------------------------------------------------------')
        return df
    
    except pd.io.common.ParserError:
        print("Dados não lidos")

def real_float(df, col):
    """ Método para modificar as colunas do tipo money para float e salvar o arquivo num caminho.
    
    Args: \n
        df: Dataframe que terá a coluna passada modificada. \n
        col: Lista com a(s) coluna(s) que será(ão) modificada(s). \n
        path: Caminho onde será salvo o novo dataframe com a coluna modificada.
    Returns: \n
        Retorna um dataframe com uma coluna modificada do tipo money para float.
    """
    for i in col:
        df[i] = [x.replace('R$ ', '') for x in df[i]]
        df[i] = [x.replace('.', '') for x in df[i]]
        df[i] = [x.replace(',', '.') for x in df[i]]
        df[i] = df[i].astype(float)
    return df  
    # df.to_csv(path, index=False)

# Lendo os arquivos do co.csv
# co = readfile('/content/co_202008281640_copy.csv')

# converter a coluna contraprestação para float e salvando no diretorio do colab
# col = ['contraprestacao']
# co_float = real_float(co, col, '/content/novo_667_co.csv')

# Lendo os arquivos do si.csv
si = pd.read_csv('/content/si_202008281640.csv')
col = ['despesa']
si_float = real_float(si, col)
si_float = si_float.fillna(0)
si_float.mat_secund = si_float.mat_secund.astype('int')
si_float.num_diarias = si_float.num_diarias.astype('int')
si_float.carencia = si_float.carencia.astype('int')
si_float.to_csv('/content/novo_si.csv', index=False)